In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

#for working on time and dates
import datetime as dt
import time

#yahoo finance lib
import yfinance as yf

#for handling files
import os 
from os import listdir
from os.path import isfile, join

#import plotly lib
import cufflinks as cf
import plotly as ply
import plotly.express as px
import plotly.graph_objects as go

#Make Plotly work in jupyter notebook
ply.offline.init_notebook_mode(connected=True)
    #ply.offline(parameter) connected: Literal[True] ook_mode, plot, iplot
    #init_notebook_mode(connected=True)

#use plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter('ignore')

## Constants

In [2]:
Path_5y = "C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/5y/"
Path_1y = "C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/1y/"
Path_6mo = "C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/6mo/"
Path_3mo = "C:/Users/davin/Documents/Data Science/My Projects/Untitled Folder/My Portfolio/3mo/"

#start end date defaults
s_date = '2017-02-01'
e_date = '2022-12-06'
s_date_dt = pd.to_datetime(s_date)
e_date_dt = pd.to_datetime(e_date)

## Get Dataframe from CSV

In [3]:
def get_stock_df_from_csv(ticker,path):
    try:
        df = pd.read_csv(path + ticker + '.csv', index_col=0)
    except FileNotFoundError as ex:
        Print('File Not Found')
    else:
        return df

## Plot Bollinger Bands

In [8]:
def plot_with_boll_bands(df,ticker,term):
    fig = go.Figure()
    
    candle = go.Candlestick(x=df.index, open = df['Open'], 
                            high = df['High'], low = df['Low'], 
                            close = df['Close'], name = 'Candlestick')
    upper_line = go.Scatter(x=df.index, y = df[term + '_Upper_band'], 
                            line = dict(color = 'rgba(0, 250, 0, 0.75)', width = 1), 
                            name = 'Upper Band')
    middle_line = go.Scatter(x=df.index, y = df[term + '_Middle_band'], 
                            line = dict(color = 'rgba(0, 0, 250, 0.75)', width = 1), 
                            name = 'Middle Band')
    lower_line = go.Scatter(x=df.index, y = df[term + '_Lower_band'], 
                            line = dict(color = 'rgba(250, 0, 0, 0.75)', width = 1), 
                            name = 'Lower Band')
    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(middle_line)
    fig.add_trace(lower_line)
    
    fig.update_xaxes(title='Date', rangeslider_visible = True)
    fig.update_yaxes(title='Price')
    
    fig.update_layout(title = ticker + ' Bollinger bands', height = 600, width = 800, 
                      showlegend = True)
    fig.show()

In [10]:
df = get_stock_df_from_csv("BBCA_JK",Path_1y)
plot_with_boll_bands(df,'BCA','S')

## Plot Ichimoku

In [ ]:
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

In [ ]:
def get_ichimoku(df,ticker):
    candle = go.Candlestick(x=df.index, open = df['Open'], 
                            high = df['High'], low = df['Low'], 
                            close = df['Close'], name = 'Candlestick')
    df1 = df.copy()
    fig = go.Figure()
    df['label'] = np.where(df['SpanA']> df['SpanB'],1, 0)
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()
    
    df = df.groupby('group')
    
    dfs = []
    for name, data in df:
        dfs.append(data)
    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index, y = df.SpanA, 
                                  line = dict(color = 'rgba(0, 0, 0, 0)')))
        fig.add_traces(go.Scatter(x=df.index, y = df.SpanB,
                                  line = dict(color = 'rgba(0, 0, 0, 0)'),
                                  fill = 'tonexty',
                                  fillcolor = get_fill_color(df['label'].iloc[0])))
    
    baseline = go.Scatter(x = df1.index, y = df1['Baseline'],
                          line = dict(color = 'pink',width = 2),
                          name = 'baseline')    
    conversion = go.Scatter(x = df1.index, y = df1['Conversion'],
                            line = dict(color = 'black',width = 2),
                            name = 'Conversion')
    lagging = go.Scatter(x = df1.index, y = df1['Lagging'],
                            line = dict(color = 'purple',width = 2),
                            name = 'Lagging')
    span_a = go.Scatter(x = df1.index, y = df1['SpanA'],
                            line = dict(color = 'green',width = 2, dash = 'dot'),
                            name = 'Span A')
    span_b = go.Scatter(x = df1.index, y = df1['SpanB'],
                            line = dict(color = 'red',width = 2),
                            name = 'SpanB')
    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lagging)
    fig.add_trace(span_a)
    fig.add_trace(span_b)
    
    fig.update_layout(title = ticker + ' Ichimoku', height = 600, width = 800, 
                      showlegend = True)
    fig.show()

In [ ]:
get_ichimoku(df, 'BCA')

## Get Sector Stocks